In [21]:
import os
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import os
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%load_ext autoreload
%autoreload 2

In [18]:
path_to_model = 'frozen_inference_graph_coco.pb'
path_to_config= 'mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'
SAMPLE_IMAGE='dog1.png'
SAMPLE_MASK="mask.png"
ALGO_INPAINT="algo_inpaint.png"
MODEL_INPAINT="model_inpaint.png"
BATCH_SIZE = 4
colors = np.random.randint(125, 255, (80, 3))
IMAGE_INPAINT = 15

# PixelLib

In [8]:
model = cv2.dnn.readNetFromTensorflow(path_to_model,path_to_config)
img = cv2.imread(SAMPLE_IMAGE)
img = cv2.resize(img,(650,550))
original_img = cv2.imread(SAMPLE_IMAGE)
original_img = cv2.resize(img,(650,550))
height, width, _ = img.shape
black_image = np.zeros((height, width, 3), np.uint8)
black_image[:] = (0, 0, 0)

In [9]:
blob = cv2.dnn.blobFromImage(img, swapRB=True)
model.setInput(blob)
boxes, masks = model.forward(["detection_out_final", "detection_masks"])
detection_count = boxes.shape[2]

In [10]:
for i in range(detection_count):
     box = boxes[0, 0, i]
     class_id = box[1]
     score = box[2]
     if score < 0.5:
         continue
     x = int(box[3] * width)
     y = int(box[4] * height)
     x2 = int(box[5] * width)
     y2 = int(box[6] * height)
     roi = black_image[y: y2, x: x2]
     roi_height, roi_width, _ = roi.shape
     mask = masks[i, int(class_id)]
     mask = cv2.resize(mask, (roi_width, roi_height))
     _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)
     cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 3)
     contours, _ = cv2.findContours(np.array(mask, np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
     color = colors[int(class_id)]
     for cnt in contours:
         cv2.fillPoly(roi, [cnt], (int(color[0]), int(color[1]), int(color[2])))

cv2.imwrite('output.png', np.hstack([img,black_image]))
cv2.imwrite('Overlay_image.png', ((0.6*black_image)+(0.4*img)).astype("uint8"))

True

In [11]:
onebite_masc = cv2.cvtColor(black_image, cv2.COLOR_BGR2GRAY)


In [19]:
img_pre_output = original_img.copy()

j = 0
for row in img_pre_output:
    i = 0
    for element in row:
        if (onebite_masc[j, i] != 0):
            row[i] = [255, 255, 255]
        i = i + 1
    # print(row)
    j = j + 1

img_output = original_img.copy()

j = 0
for row in img_pre_output:
    i = 0
    for element in row:
        if np.array_equal(element, [255, 255, 255]):
                img_output = cv2.circle(img_output, (i, j), IMAGE_INPAINT, (255, 255, 255), 0)
        i = i + 1
    j = j + 1
    
mask_output = img_output.copy()

j = 0
for row in mask_output:
    i = 0
    for element in row:
        if not np.array_equal(element, [255, 255, 255]):
            row[i] = [0, 0, 0]
        i = i + 1
    # print(row)
    j = j + 1

In [13]:
print(mask_output.shape)

(550, 650, 3)


In [20]:
final_onebite_masc = cv2.cvtColor(mask_output, cv2.COLOR_BGR2GRAY)
cv2.imwrite(SAMPLE_MASK, final_onebite_masc)
dst = cv2.inpaint(original_img,final_onebite_masc,3,cv2.INPAINT_TELEA)
output_mask_img = cv2.add(original_img,mask_output)
cv2.imwrite(ALGO_INPAINT, dst)
cv2.imwrite('output_mask_img .png', output_mask_img )

True